# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# pull csv
city_df = pd.read_csv('Images/cities.csv')
city_df.head()




,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Agadez,19.75,10.25,85.06,13,0,3.94,NE,1603127368
1,1,Gayéri,12.65,0.49,97.14,24,50,4.23,BF,1603127368
2,2,Hollola,61.05,25.43,33.01,97,100,1.99,FI,1603127368
3,3,Gizo Government Station,-8.10,156.84,81.95,74,25,2.06,SB,1603127368
4,4,Broken Hill,-31.95,141.43,53.60,54,99,6.93,AU,1603127369


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)


In [4]:
# location and humidity variables from df
locations = city_df[['Lat','Lng']].astype(float)
heat = city_df['Humidity'].astype(float)

In [5]:

# create heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = heat, dissipating=False, max_intensity = 100,
                                point_radius = 3)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# narrow down df to ideal criteria
temp = (city_df['Max Temp'] > 70) & (city_df['Max Temp'] < 83)
wind = city_df['Wind Speed'] < 10
humidity = city_df['Humidity'] < 65

# combine
ideal = temp & wind & humidity

# create new df
ideal_df = city_df[ideal]
ideal_df = ideal_df.dropna()

# display check
ideal_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,Asilah,35.47,-6.03,73.4,60,54,2.24,MA,1603127375
68,68,Kapaa,22.08,-159.32,75.0,59,1,1.01,US,1603127226
115,115,Kapaa,22.08,-159.32,75.0,59,1,1.01,US,1603127226
126,126,Kapaa,22.08,-159.32,75.0,59,1,1.01,US,1603127226
135,135,Guerrero Negro,27.98,-114.06,79.2,56,0,5.35,MX,1603127395


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create hotl column in df
ideal_df['Hotel Name'] = ""
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,34,Asilah,35.47,-6.03,73.40,60,54,2.24,MA,1603127375,
68,68,Kapaa,22.08,-159.32,75.00,59,1,1.01,US,1603127226,
115,115,Kapaa,22.08,-159.32,75.00,59,1,1.01,US,1603127226,
126,126,Kapaa,22.08,-159.32,75.00,59,1,1.01,US,1603127226,
135,135,Guerrero Negro,27.98,-114.06,79.20,56,0,5.35,MX,1603127395,
141,141,Nizwá,22.93,57.53,76.53,23,72,1.90,OM,1603127340,
149,149,Sagauli,26.78,84.73,75.94,57,18,4.43,IN,1603127397,
190,190,Dillon,34.42,-79.37,75.99,53,1,4.70,US,1603127407,
203,203,Kapaa,22.08,-159.32,75.00,59,1,1.01,US,1603127226,
267,267,Dumka,24.27,87.25,76.86,61,0,1.30,IN,1603127422,


In [8]:
# url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set params to seaarch for hotels within 5k meters
params = {
    'radius': 5000,
    'types': 'hotels',
    'keyword': 'hotel',
    'key': g_key
}

# for loop to commence search
for index, row in ideal_df.iterrows():
    
    # get lat and long coordinates
    lat = row['Lat']
    lng = row['Lng']
    
    # city
    city = row['City']
    
    # location param
    params['location'] = f'{lat},{lng}'
    
    # get url and prep it (json)
    print(f'Retrieving results for {index}: {city}')
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
          
    try:
        
        ideal_df.loc[index, 'Hotel Name'] = results[0]['name']
          
    except (KeyError, IndexError):
         
        print(f'No hotel found in 5k radius of {city}. Proceeding to search next city...')
          
    print('----------------------------------')

Retrieving results for 34: Asilah
----------------------------------
Retrieving results for 68: Kapaa
----------------------------------
Retrieving results for 115: Kapaa
----------------------------------
Retrieving results for 126: Kapaa
----------------------------------
Retrieving results for 135: Guerrero Negro
----------------------------------
Retrieving results for 141: Nizwá
----------------------------------
Retrieving results for 149: Sagauli
No hotel found in 5k radius of Sagauli. Proceeding to search next city...
----------------------------------
Retrieving results for 190: Dillon
----------------------------------
Retrieving results for 203: Kapaa
----------------------------------
Retrieving results for 267: Dumka
----------------------------------
Retrieving results for 371: Chui
----------------------------------
Retrieving results for 379: Silver City
----------------------------------
Retrieving results for 435: Riyadh
----------------------------------
Retrieving r

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]


In [10]:
# Add marker layer ontop of heat map
heatmap = {
    'width': '1000px',
    'height': '500px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout = heatmap, zoom_level = 2, center = (0,0))

hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# save fig
plt.savefig('Images/hotel_map.png')

# Display figure
fig

Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>